In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@error-handling

# Experiment Outputs for testing


In [2]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler
base_folder = colab.init('My Drive')
config_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(config_path)

2024-07-28 21:33:19 - INFO - PyTorch version 1.13.1 available.
2024-07-28 21:33:21 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Data-Extraction-Phase/notebooks/My Drive
2024-07-28 21:33:21 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [3]:
import torch
model_name='arabertv02'
data_name='ANERCorp_CamelLab'
training_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

load_model_path = fh.file_path / f"trainOutputs/{model_name}_{data_name}_regular.bin"
model = torch.load(load_model_path, map_location=torch.device('cpu'))

In [4]:
from experiment_utils.model_outputs import ModelOutputWorkflowManager
from experiment_utils.tokenization import TokenizationWorkflowManager
from experiment_utils.evaluation import Metrics

In [5]:
results_dict = {
            "token_results": training_outputs.test_metrics.skl_results,
            "token_report": training_outputs.test_metrics.skl_report,
            "token_outputs": training_outputs.test_metrics.skl_output,
            "entity_results": training_outputs.test_metrics.seq_results,
            "entity_report": training_outputs.test_metrics.seq_report,
            "entity_outputs": training_outputs.test_metrics.seq_output
        }

results = Metrics.from_dict(results_dict)

In [6]:
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
corpora_fh = FileHandler(corpora_path)
corpora  = corpora_fh.load_json('corpora.json')

config = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
config_fh = FileHandler(config)
config = config_fh.load_yaml('config.yaml')

In [7]:
from experiment_utils.train import DatasetManager
from experiment_utils.configurations import TrainingConfig, TokenizationConfig
split = 'test'
args = TrainingConfig.from_dict(config.get('training'))
tokenization_config = TokenizationConfig.from_dict(config.get('tokenization'))
data_manager = DatasetManager(corpora, 'ANERCorp_CamelLab', tokenization_config)
batch_sizes = {'train': args.train_batch_size, 'test': args.test_batch_size, 'validation': args.test_batch_size}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-07-28 21:33:24 - INFO - Configuration validated successfully


In [8]:
NERData = corpora['ANERCorp_CamelLab']
tokenization_outputs = TokenizationWorkflowManager(NERData, tokenization_config)

2024-07-28 21:33:24 - INFO - Loading Tokenizer aubmindlab/bert-base-arabertv02


2024-07-28 21:33:24 - INFO - Loading Preprocessor aubmindlab/bert-base-arabertv02
2024-07-28 21:33:24 - INFO - Processing train split


  0%|          | 0/4149 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model_outputs = ModelOutputWorkflowManager(model, device, data_manager, batch_sizes, 'test')

2024-07-27 23:24:11 - INFO - Specific Split test being processed
2024-07-27 23:24:11 - INFO - Loading Preprocessor: aubmindlab/bert-base-arabertv02
2024-07-27 23:24:11 - INFO - Loading Tokenizer: aubmindlab/bert-base-arabertv02, lower_case: False


  0%|          | 0/121 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from collections import defaultdict
from dataclasses import dataclass, field

import numpy as np

class LabelAligner:
    def __init__(self, predictions, tokenized_sentences):
        """
        Initialize the DataPreparation class.

        Args:
            batches: The batches object containing batch data.
            tokenization_outputs: The tokenization outputs object.
        """
        self.tokenized_sentences = tokenized_sentences
        self.predictions = predictions

    def get_alignment_locations(self):
        """
        Create a map for label alignment based on tokenization outputs.

        Returns:
            alignment_map: A dictionary mapping sentence IDs to token indices and tokens.
        """
        alignment_locations = defaultdict(list)
        for sentence_id, sentence in enumerate(self.tokenized_sentences):
            for token_label_id, token_label in enumerate(sentence.labels_df):
                if token_label in ["[CLS]", "[SEP]", "IGNORED"]:
                    alignment_locations[sentence_id].append((token_label_id, token_label))
        return alignment_locations

    def align_labels(self):
        """
        Modify predictions based on the alignment map.

        Args:
            preds: A list of predictions.
            pred_map: A dictionary mapping sentence IDs to token indices and tokens.

        Returns:
            modified_preds: A list of modified predictions.
        """
        alignment_locations = self.get_alignment_locations()
        modified_predictions = []
        for sentence_id, original_sentence in enumerate(self.predictions):
            sentence = original_sentence[:]  # Create a shallow copy of the list
            for index, token in alignment_locations[sentence_id]:
                # no need to shift as the indices were calculated according to the tokenized version in the truth labels
                sentence.insert(index, token)
            modified_predictions.append(sentence)
        return modified_predictions



@dataclass
class AnalysisData:
    model_outputs: list
    tokenization_outputs: list
    last_hidden_states: torch.Tensor = field(init=False)
    labels: torch.Tensor = field(init=False)
    losses: torch.Tensor = field(init=False)
    token_ids: torch.Tensor = field(init=False)
    words: list = field(init=False)
    tokens: list = field(init=False)
    word_pieces: list = field(init=False)
    core_tokens: list = field(init=False)
    true_labels: list = field(init=False)
    pred_labels: list = field(init=False, default_factory=list)
    sentence_ids: list = field(init=False)
    token_positions: list = field(init=False)
    token_selector_id: list = field(init=False)
    agreements: list = field(init=False)
    x: list = field(init=False)
    y: list = field(init=False)
    
    def __post_init__(self):
        self.last_hidden_states = torch.concat([s.last_hidden_states for s in self.model_outputs])
        self.labels = torch.concat([s.labels for s in self.model_outputs])
        self.losses = torch.concat([s.losses for s in self.model_outputs])
        self.token_ids = torch.concat([s.input_ids for s in self.model_outputs])
        self.words = [word for sentence in self.tokenization_outputs for word in sentence.words_df]
        self.tokens = [token for sentence in self.tokenization_outputs for token in sentence.tokens_df]
        self.word_pieces = [wp for sentence in self.tokenization_outputs for wp in sentence.word_pieces_df]
        self.core_tokens = [ct for sentence in self.tokenization_outputs for ct in sentence.core_tokens_df]
        self.true_labels = [label for sentence in self.tokenization_outputs for label in sentence.labels_df]
        self.sentence_ids = [index for sentence in self.tokenization_outputs for index in sentence.sentence_index_df]
        self.token_positions = [position for sentence in self.tokenization_outputs for position in range(len(sentence.tokens_df))]
        self.token_selector_id = [
            f"{core_token}@#{token_position}@#{sentence_index}" 
            for core_token, token_position, sentence_index in
            zip(self.core_tokens, self.token_positions, self.sentence_ids)
        ]
        self.agreements = np.array(self.true_labels) == np.array(self.pred_labels)
    
    def align_labels(self, aligner):
        aligned_labels = aligner.align_labels()
        self.pred_labels = [label for sentence in aligned_labels for label in sentence]
    
    def assign_coordinate(self, coordinates):
        self.x = coordinates[0]
        self.y = coordinates[1]
        
    def to_dict(self):
        analysis_data = {
            "labels": self.labels,
            "losses": self.losses,
            "token_ids": self.token_ids,
            "words": self.words,
            "tokens": self.tokens,
            "word_pieces": self.word_pieces,
            "core_tokens": self.core_tokens,
            "true_labels": self.true_labels,
            "pred_labels": self.pred_labels,
            "sentence_ids": self.sentence_ids,
            "token_positions": self.token_positions,
            "token_selector_id": self.token_selector_id,
            "agreements": self.agreements,
            "x": self.x,
            "y": self.y
        }
        return analysis_data
        
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class UMAPConfig:
    n_neighbors: int = 15
    min_dist: float = 0.1
    metric: str = "cosine"
    random_state: int = 1
    verbose: bool = True
    normalize_embeddings: bool = False

    def set_params(self, 
                   n_neighbors: Optional[int] = None, 
                   min_dist: Optional[float] = None, 
                   metric: Optional[str] = None, 
                   normalize_embeddings: Optional[bool] = None):
        """Optionally update UMAP parameters."""
        if n_neighbors is not None:
            self.n_neighbors = n_neighbors
        if min_dist is not None:
            self.min_dist = min_dist
        if metric is not None:
            self.metric = metric
        if normalize_embeddings is not None:
            self.normalize_embeddings = normalize_embeddings

    @staticmethod
    def from_dict(config_dict):
        """Create UMAPConfig from a dictionary."""
        return UMAPConfig(**config_dict)

    def __post_init__(self):
        """Validate UMAP configuration to ensure valid settings."""
        if not isinstance(self.n_neighbors, int) or self.n_neighbors <= 0:
            raise ValueError("n_neighbors must be a positive integer.")
        if not isinstance(self.min_dist, float) or self.min_dist < 0:
            raise ValueError("min_dist must be a non-negative float.")
        if not isinstance(self.metric, str):
            raise ValueError("metric must be a string.")
        if not isinstance(self.verbose, bool):
            raise ValueError("verbose must be a boolean.")
        if not isinstance(self.normalize_embeddings, bool):
            raise ValueError("normalize_embeddings must be a boolean.")


class DataExtractor:
    def __init__(self, tokenization_outputs, model_outputs, results, transformer):
        """
        Initialize the FlatDataExtractor class.

        Args:
            preparation: The DataPreparation object.
            outputs: The outputs object containing batch outputs.
            results: The results object containing model results.
        """
        self.aligner = LabelAligner(results.entity_outputs['y_pred'].copy(), tokenization_outputs)
        self.tokenization_outputs = tokenization_outputs
        self.model_outputs = model_outputs
        self.transformer = transformer

    def extract_flat_data(self):
        """
        Extract and flatten the data from the preparation, outputs, and results.

        Returns:
            A tuple of flattened data elements.
        """
        
        
        self.analysis_data = AnalysisData(
            tokenization_outputs=self.tokenization_outputs,
            model_outputs=self.model_outputs,
        )
        # analysis_data.align_labels(self.aligner)
        # coordinates = self.transformer.apply_umap(analysis_data.last_hidden_states)
        # analysis_data.assign_coordinate(coordinates)
        # return analysis_data.to_dict()
        
        # flat_last_hidden_states = torch.concat(
        #     [
        #         sentence.last_hidden_states for sentence in self.model_outputs
        #     ]
        #     )

        # flat_labels = torch.concat(
        #     [
        #         sentence.labels for sentence in self.model_outputs
        #     ]
        #     )


        # flat_losses = torch.concat(
        #     [
        #         sentence.losses for sentence in self.model_outputs
        #     ]
        #     )

        # flat_token_ids = torch.concat(
        #     [
        #         sentence.input_ids for sentence in self.model_outputs
        #     ]
        #     )


        # flat_words = [
        #     word for sentence in self.tokenization_outputs
        #     for word in sentence.words_df
        # ]

        # flat_tokens = [
        #     token for sentence in self.tokenization_outputs
        #     for token in sentence.tokens_df
        # ]

        # flat_word_pieces = [
        #     word_piece for sentence in self.tokenization_outputs
        #     for word_piece in sentence.word_pieces_df
        # ]

        # flat_core_tokens = [
        #     core_token for sentence in self.tokenization_outputs
        #     for core_token in sentence.core_tokens_df
        # ]

        # flat_true_labels = [
        #     label for sentence in self.tokenization_outputs
        #     for label in sentence.labels_df
        # ]

        # flat_pred_labels = [
        #     label for sentence in self.aligner.align_labels()
        #     for label in sentence
        # ]

        # flat_sentence_ids = [
        #     sentence_index for sentence in self.tokenization_outputs
        #     for sentence_index in sentence.sentence_index_df
        # ]

        # flat_token_positions = [
        #     position for sentence in self.tokenization_outputs
        #     for position in range(len(sentence.tokens_df))
        # ]

        # flat_token_selector_id = [
        #     f"{core_token}@#{token_position}@#{sentence_index}" 
        #     for core_token, token_position, sentence_index in
        #     zip(flat_core_tokens, flat_token_positions, flat_sentence_ids)
        # ]

        # flat_agreements = np.array(flat_true_labels) == np.array(flat_pred_labels)
        # analysis_data = {
        #     "flat_labels": flat_labels,
        #     "flat_losses": flat_losses,
        #     "flat_token_ids": flat_token_ids,
        #     "flat_words": flat_words,
        #     "flat_tokens": flat_tokens,
        #     "flat_word_pieces": flat_word_pieces,
        #     "flat_core_tokens": flat_core_tokens,
        #     "flat_true_labels": flat_true_labels,
        #     "flat_pred_labels": flat_pred_labels,
        #     "flat_sentence_ids": flat_sentence_ids,
        #     "flat_token_positions": flat_token_positions,
        #     "flat_token_selector_id": flat_token_selector_id,
        #     "flat_agreements": flat_agreements
        # }


        # return flat_last_hidden_states, analysis_data



In [122]:
umap_config = UMAPConfig.from_dict(config.get('umap'))


In [123]:
from umap import UMAP
from sklearn.preprocessing import normalize

from umap import UMAP
from sklearn.preprocessing import normalize

class DataTransformer:
    def __init__(self, umap_config):
        """
        Initialize the DataTransformer with configuration for UMAP.
        """
        self.umap_config = umap_config

    def apply_umap(self, data):
        """
        Apply UMAP dimensionality reduction to the given data.
        """
        if self.umap_config.normalize_embeddings:
            data = normalize(data, axis=1)
        umap_model = UMAP(
            n_neighbors=self.umap_config.n_neighbors,
            min_dist=self.umap_config.min_dist,
            metric=self.umap_config.metric,
            random_state=self.umap_config.random_state,
            verbose=self.umap_config.verbose,
        )
        return umap_model.fit_transform(data).transpose()
        # return umap_model 




# class DataTransformer:
#     def __init__(self, umap_config, flat_states):
#         """
#         Initialize the DataTransformer class with UMAP parameters and normalization option.

#         Args:
#             umap_config: configuration of umap paramaters
#         """
#         self.flat_states = flat_states

#         self.umap_model = UMAP(
#             n_neighbors=umap_config.n_neighbors,
#             min_dist=umap_config.min_dist,
#             metric=umap_config.metric,
#             random_state=umap_config.random_state,
#             verbose=umap_config.verbose,
#         )
#         self.normalize_embeddings = umap_config.normalize_embeddings

#     def apply_umap(self):
#         """
#         Apply UMAP dimensionality reduction to the given flat states.

#         Args:
#             flat_states: The high-dimensional data to reduce.

#         Returns:
#             The reduced data.
#         """
#         if self.normalize_embeddings:
#             flat_states = normalize(self.flat_states, axis=1)
#         return self.umap_model.fit_transform(flat_states).transpose()
    
    

#     @staticmethod
#     def transform_to_dataframe(self):
#         coordinates = self.apply_umap()
#         return coordinates

In [124]:
transformer = DataTransformer(umap_config)


In [125]:
analysis_data_extractor = DataExtractor(tokenization_outputs.test, model_outputs.test, results, transformer)

In [126]:
analysis_data_extractor.extract_flat_data()

/var/folders/64/7h5mlrd57hg7h3l2cjpzwz7r000zbz/T/ipykernel_15468/1856640133.py:93: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  self.agreements = np.array(self.true_labels) == np.array(self.pred_labels)
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


UMAP(angular_rp_forest=True, metric='cosine', random_state=1, verbose=True)
Wed Jul 24 21:14:03 2024 Construct fuzzy simplicial set
Wed Jul 24 21:14:03 2024 Finding Nearest Neighbors
Wed Jul 24 21:14:03 2024 Building RP forest with 14 trees
Wed Jul 24 21:14:08 2024 NN descent for 15 iterations


: 

In [132]:
aligner = LabelAligner(results, tokenization_outputs.test['tokenized_text'])

In [62]:
analysis_df['losses']

0        0.000000
1        0.305853
2        0.021982
3        0.001037
4        0.022747
           ...   
29706    0.000000
29707    0.005826
29708    0.003222
29709    0.000037
29710    0.000000
Name: losses, Length: 29711, dtype: float64

In [100]:
len(label_aligner.change_preds(results.entity_outputs['y_pred'])[0])

49

In [139]:
tokenization_outputs.test['tokenized_text'][0].labels_df

['[CLS]',
 'B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'IGNORED',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [136]:
results.entity_outputs['y_pred'][0]

['B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [135]:
output[0]

['[CLS]',
 'B-LOC',
 'B-LOC',
 'O',
 'B-PERS',
 'I-PERS',
 'IGNORED',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'B-PERS',
 'I-PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'B-LOC',
 'IGNORED',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '[SEP]']

In [19]:
import pandas as pd


analysis_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_analysis_df.jsonl.gz',
    lines=True
    )
# seqeval calculations for entity confusion matrix
confusion_data = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_confusion_data.jsonl.gz',
    lines=True
    )


controid_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_controid_df.jsonl.gz',
    lines=True
    )


entity_prediction = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_entity_prediction.jsonl.gz',
    lines=True
    )


# this is just the x and y coordinates of the pretrained model
pretrained_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_pre_df.jsonl.gz',
    lines=True
    )
# this is exact same information but with extra truth and pred score
token_score_df = pd.read_json(
    '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Class Imbalance/3_errorAnalysis/Analysis/ANERCorp_CamelLab/test/test_token_score_df.jsonl.gz',
    lines=True
    )

# Think about away of loading weights and activations
# think about where the auxilary stuff should be?

In [20]:
analysis_df.columns

Index(['global_id', 'token_id', 'word_id', 'sen_id', 'token_ids', 'label_ids',
       'first_tokens_freq', 'first_tokens_consistency',
       'first_tokens_inconsistency', 'words', 'wordpieces', 'tokens',
       'first_tokens', 'truth', 'pred', 'agreement', 'losses', 'x', 'y',
       'tokenization_rate', 'token_entropy', 'word_entropy', 'tr_entity',
       'pr_entity', 'error_type', 'prediction_entropy', 'confidences',
       'variability', 'O', 'B-PERS', 'I-PERS', 'B-ORG', 'I-ORG', 'B-LOC',
       'I-LOC', 'B-MISC', 'I-MISC', '3_clusters', '4_clusters', '9_clusters'],
      dtype='object')

In [40]:
controid_df

,token_ids,truth,pred,agreement,error_type,centroid,clusters,x,y
0,الصالحية@#0@#1,B-LOC,B-LOC,True,Correct,Centroid-3,cluster-2,7.458792,-4.840983
1,المفرق@#0@#2,B-LOC,B-LOC,True,Correct,Centroid-3,cluster-2,7.550399,-4.857412
2,-@#0@#3,O,O,True,Correct,Centroid-3,cluster-1,1.931367,17.854967
3,غيث@#0@#4,B-PERS,B-PERS,True,Correct,Centroid-3,cluster-0,-4.481947,12.181932
4,الطر@#0@#5,I-PERS,I-PERS,True,Correct,Centroid-3,cluster-0,-6.245377,9.296581
...,...,...,...,...,...,...,...,...,...
74984,C,C,C,C,C,Centroid-9,C,10.065314,-6.042493
74985,C,C,C,C,C,Centroid-9,C,13.743145,-4.104878
74986,C,C,C,C,C,Centroid-9,C,0.448427,3.712907
74987,C,C,C,C,C,Centroid-9,C,1.370305,1.797442


In [36]:
analysis_df['pred'].iloc[50:100]

50          O
51    IGNORED
52          O
53          O
54          O
55          O
56          O
57          O
58          O
59          O
60          O
61          O
62          O
63          O
64          O
65          O
66          O
67          O
68          O
69          O
70          O
71          O
72      B-LOC
73      B-LOC
74    IGNORED
75          O
76          O
77          O
78          O
79    IGNORED
80    IGNORED
81          O
82          O
83          O
84      [SEP]
85      [CLS]
86          O
87          O
88          O
89          O
90          O
91          O
92          O
93          O
94          O
95          O
96    IGNORED
97          O
98          O
99          O
Name: pred, dtype: object

In [30]:
len(analysis_df['truth'])

29711

In [23]:
analysis_df[['tr_entity',
       'pr_entity', 'error_type']]

,tr_entity,pr_entity,error_type
0,[CLS],[CLS],Correct
1,LOC,LOC,Correct
2,LOC,LOC,Correct
3,O,O,Correct
4,PERS,PERS,Correct
...,...,...,...
29706,IGNORED,IGNORED,Correct
29707,PERS,PERS,Correct
29708,PERS,PERS,Correct
29709,O,O,Correct


In [32]:
confusion_data.head(20)

,truth,pred
0,LOC,LOC
1,LOC,LOC
2,PERS,PERS
3,PERS,PERS
4,LOC,LOC
5,LOC,LOC
6,PERS,O
7,LOC,LOC
8,LOC,LOC
9,O,LOC
